# Strands Agents에 사용자 정의 도구 추가하기

## 개요
이 예제에서는 Strands Agents를 사용하여 사용자 정의 도구를 만드는 다양한 방법을 안내합니다. 로컬 SQLite 데이터베이스에 연결하여 데이터 작업을 수행하는 개인 비서 사용 사례를 구축하겠습니다. 보너스로 `BedrockModel` 클래스의 `thinking` 필드를 사용하여 Claude Sonnet 3.7의 추론 기능 사용법도 안내하겠습니다.

## 에이전트 세부사항
<div style="float: left; margin-right: 20px;">
    
|기능                |설명                                               |
|--------------------|---------------------------------------------------|
|사용된 네이티브 도구 |current_time, calculator                           |
|생성된 사용자 정의 도구|create_appointment, list_appointments              |
|에이전트 구조        |단일 에이전트 아키텍처                              |

</div>


## 아키텍처

<div style="text-align:left">
    <img src="images/architecture.png" width="85%" />
</div>

## 주요 기능
* **단일 에이전트 아키텍처**: 이 예제는 내장 도구와 사용자 정의 도구와 상호작용하는 단일 에이전트를 생성합니다
* **내장 도구**: Strands Agent의 도구 사용법을 배웁니다
* **사용자 정의 도구**: 자신만의 도구를 만드는 방법을 배웁니다
* **기본 LLM으로서의 Bedrock 모델**: 기본 LLM 모델로 Amazon Bedrock의 Anthropic Claude 3.7을 사용했습니다

## 설정 및 사전 요구사항

### 사전 요구사항
* Python 3.10+
* AWS 계정
* Amazon Bedrock에서 Anthropic Claude 3.7 활성화, [가이드](https://docs.aws.amazon.com/bedrock/latest/userguide/model-access.html)
* Amazon Bedrock Knowledge Base, Amazon S3 버킷 및 Amazon DynamoDB를 생성할 권한이 있는 IAM 역할

이제 Strands Agent에 필요한 패키지들을 설치하겠습니다

In [ ]:
# 사전 요구사항 설치
%pip install -r requirements.txt

### 의존성 패키지 가져오기

이제 의존성 패키지들을 가져오겠습니다

In [ ]:
import json
import sqlite3
import uuid
from datetime import datetime

from strands import Agent, tool
from strands.models import BedrockModel

## 사용자 정의 도구 정의
다음으로 로컬 SQLite 데이터베이스와 상호작용하기 위한 사용자 정의 도구를 정의해보겠습니다:
* **create_appointment**: 고유 ID, 날짜, 위치, 제목 및 설명으로 새 개인 약속 생성
* **list_appointment**: 사용 가능한 모든 약속 나열
* **update_appointments**: 약속 ID를 기반으로 약속 업데이트

### 에이전트와 같은 파일에서 도구 정의하기

Strands Agents SDK로 도구를 정의하는 방법은 여러 가지가 있습니다. 첫 번째 방법은 함수에 `@tool` 데코레이터를 추가하고 문서를 제공하는 것입니다. 이 경우 Strands Agents는 함수 문서, 타이핑 및 인수를 사용하여 에이전트에 도구를 제공합니다. 이 경우 에이전트와 같은 파일에서 도구를 정의할 수도 있습니다

In [ ]:
@tool
def create_appointment(date: str, location: str, title: str, description: str) -> str:
    """
    데이터베이스에 새 개인 약속을 생성합니다.

    Args:
        date (str): 약속 날짜 및 시간 (형식: YYYY-MM-DD HH:MM).
        location (str): 약속 장소.
        title (str): 약속 제목.
        description (str): 약속 설명.

    Returns:
        str: 새로 생성된 약속의 ID.

    Raises:
        ValueError: 날짜 형식이 잘못된 경우.
    """
    # 날짜 형식 검증
    try:
        datetime.strptime(date, "%Y-%m-%d %H:%M")
    except ValueError:
        raise ValueError("날짜는 'YYYY-MM-DD HH:MM' 형식이어야 합니다")

    # 고유 ID 생성
    appointment_id = str(uuid.uuid4())

    conn = sqlite3.connect("appointments.db")
    cursor = conn.cursor()

    # appointments 테이블이 없으면 생성
    cursor.execute(
        """
    CREATE TABLE IF NOT EXISTS appointments (
        id TEXT PRIMARY KEY,
        date TEXT,
        location TEXT,
        title TEXT,
        description TEXT
    )
    """
    )

    cursor.execute(
        "INSERT INTO appointments (id, date, location, title, description) VALUES (?, ?, ?, ?, ?)",
        (appointment_id, date, location, title, description),
    )

    conn.commit()
    conn.close()
    return f"ID {appointment_id}로 약속이 생성되었습니다"

### 모듈 기반 접근 방식으로 도구 정의

도구를 독립적인 파일로 정의하고 에이전트에 가져올 수도 있습니다. 이 경우에도 데코레이터 접근 방식을 사용하거나 TOOL_SPEC 딕셔너리를 사용하여 함수를 정의할 수 있습니다. 형식은 도구 사용을 위한 [Amazon Bedrock Converse API](https://docs.aws.amazon.com/bedrock/latest/userguide/tool-use-examples.html)에서 사용하는 것과 유사합니다. 이 경우 필수 매개변수와 성공 및 오류 실행의 반환값을 정의하는 데 더 유연하며 TOOL_SPEC 정의가 작동합니다.

#### 데코레이터 접근 방식

독립적인 파일에서 데코레이터를 사용하여 도구를 정의할 때, 프로세스는 에이전트와 같은 파일에서 하는 것과 매우 유사하지만, 나중에 에이전트 도구를 가져와야 합니다.

In [ ]:
%%writefile list_appointments.py
import json
import sqlite3
import os
from strands import tool

@tool
def list_appointments() -> str:
    """
    데이터베이스에서 사용 가능한 모든 약속을 나열합니다.
    
    Returns:
        str: 사용 가능한 약속들
    """
    # 데이터베이스 존재 확인
    if not os.path.exists('appointments.db'):
        return "사용 가능한 약속이 없습니다"
    
    conn = sqlite3.connect('appointments.db')
    conn.row_factory = sqlite3.Row  # 이름으로 컬럼 접근 가능
    cursor = conn.cursor()
    
    # appointments 테이블 존재 확인
    try:
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='appointments'")
        if not cursor.fetchone():
            conn.close()
            return "사용 가능한 약속이 없습니다"
        
        cursor.execute("SELECT * FROM appointments ORDER BY date")
        rows = cursor.fetchall()
        
        # 행을 딕셔너리로 변환
        appointments = []
        for row in rows:
            appointment = {
                'id': row['id'],
                'date': row['date'],
                'location': row['location'],
                'title': row['title'],
                'description': row['description']
            }
            appointments.append(appointment)
        
        conn.close()
        return json.dumps(appointments)
    
    except sqlite3.Error:
        conn.close()
        return []

#### TOOL_SPEC 접근 방식

또는 도구를 정의할 때 TOOL_SPEC 접근 방식을 사용할 수 있습니다

In [ ]:
%%writefile update_appointment.py
import sqlite3
from datetime import datetime
import os
from strands.types.tools import ToolResult, ToolUse
from typing import Any

TOOL_SPEC = {
    "name": "update_appointment",
    "description": "약속 ID를 기반으로 약속을 업데이트합니다.",
    "inputSchema": {
        "json": {
            "type": "object",
            "properties": {
                "appointment_id": {
                    "type": "string",
                    "description": "약속 ID."
                },
                "date": {
                    "type": "string",
                    "description": "약속 날짜 및 시간 (형식: YYYY-MM-DD HH:MM)."
                },
                "location": {
                    "type": "string",
                    "description": "약속 장소."
                },
                "title": {
                    "type": "string",
                    "description": "약속 제목."
                },
                "description": {
                    "type": "string",
                    "description": "약속 설명."
                }
            },
            "required": ["appointment_id"]
        }
    }
}
# 함수 이름은 도구 이름과 일치해야 함
def update_appointment(tool: ToolUse, **kwargs: Any) -> ToolResult:
    tool_use_id = tool["toolUseId"]
    appointment_id = tool["input"]["appointment_id"]
    if "date" in tool["input"]:
        date = tool["input"]["date"]
    else:
        date = None
    if "location" in tool["input"]:
        location = tool["input"]["location"]
    else:
        location = None
    if "title" in tool["input"]:
        title = tool["input"]["title"]
    else:
        title = None
    if "description" in tool["input"]:
        description = tool["input"]["description"]
    else:
        description = None
        
    # 데이터베이스 존재 확인
    if not os.path.exists('appointments.db'): 
        return {
            "toolUseId": tool_use_id,
            "status": "error",
            "content": [{"text": f"약속 {appointment_id}가 존재하지 않습니다"}]
        } 
    
    # 약속 존재 확인
    conn = sqlite3.connect('appointments.db')
    cursor = conn.cursor()
    
    # appointments 테이블 존재 확인
    try:
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='appointments'")
        if not cursor.fetchone():
            conn.close()
            return {
                "toolUseId": tool_use_id,
                "status": "error",
                "content": [{"text": f"약속 테이블이 존재하지 않습니다"}]
            }
        
        cursor.execute("SELECT * FROM appointments WHERE id = ?", (appointment_id,))
        appointment = cursor.fetchone()
        
        if not appointment:
            conn.close()
            return {
                "toolUseId": tool_use_id,
                "status": "error",
                "content": [{"text": f"약속 {appointment_id}가 존재하지 않습니다"}]
            }
        
        # 날짜 형식 검증 (제공된 경우)
        if date:
            try:
                datetime.strptime(date, '%Y-%m-%d %H:%M')
            except ValueError:
                conn.close()
                return {
                    "toolUseId": tool_use_id,
                    "status": "error",
                    "content": [{"text": "날짜는 'YYYY-MM-DD HH:MM' 형식이어야 합니다"}]
                }
        
        # 업데이트 쿼리 구성
        update_fields = []
        params = []
        
        if date:
            update_fields.append("date = ?")
            params.append(date)
        
        if location:
            update_fields.append("location = ?")
            params.append(location)
        
        if title:
            update_fields.append("title = ?")
            params.append(title)
        
        if description:
            update_fields.append("description = ?")
            params.append(description)
        
        # 업데이트할 필드가 없는 경우
        if not update_fields:
            conn.close()
            return {
                "toolUseId": tool_use_id,
                "status": "success",
                "content": [{"text": "약속을 업데이트할 필요가 없습니다. 모든 것이 설정되어 있습니다!"}]
            }
        
        # 쿼리 완성
        query = f"UPDATE appointments SET {', '.join(update_fields)} WHERE id = ?"
        params.append(appointment_id)
        
        cursor.execute(query, params)
        conn.commit()
        conn.close()
        
        return {
            "toolUseId": tool_use_id,
            "status": "success",
            "content": [{"text": f"약속 {appointment_id}가 성공적으로 업데이트되었습니다"}]
        }
    
    except sqlite3.Error as e:
        conn.close()
        return {
            "toolUseId": tool_use_id,
            "status": "error",
            "content": [{"text": str(e)}]
        }

이제 `list_appointments`와 `update_appointment`를 도구로 가져와보겠습니다

In [ ]:
import list_appointments
import update_appointment

## 에이전트 생성

사용자 정의 도구를 생성했으므로 이제 첫 번째 에이전트를 정의해보겠습니다. 이를 위해 에이전트가 해야 할 일과 하지 말아야 할 일을 정의하는 시스템 프롬프트를 생성해야 합니다. 그런 다음 에이전트의 기본 LLM 모델을 정의하고 내장 도구와 사용자 정의 도구를 제공합니다.

#### 에이전트 시스템 프롬프트 설정
시스템 프롬프트에서 에이전트에 대한 지침을 정의하겠습니다

In [ ]:
system_prompt = """당신은 내 약속과 일정 관리를 전문으로 하는 도움이 되는 개인 비서입니다.
약속 관리 도구, 계산기에 액세스할 수 있으며 현재 시간을 확인하여 내 일정을 효과적으로 정리할 수 있습니다.
필요한 경우 업데이트할 수 있도록 항상 약속 ID를 제공해주세요"""

#### 에이전트 기본 LLM 모델 정의

다음으로 에이전트의 기본 모델을 정의해보겠습니다. Strands Agents는 Amazon Bedrock 모델과 기본적으로 통합되며 모델 호출 방법을 구성할 수 있는 기능을 제공합니다. 아래에서 일부 선택적 구성이 주석 처리된 `BedrockModel` 제공자의 간단한 초기화를 볼 수 있습니다. 구성 옵션과 기본값에 대한 자세한 내용은 [Strands Agents Bedrock Model Provider 문서](https://strandsagents.com/0.1.x/user-guide/concepts/model-providers/amazon-bedrock/)에서 확인할 수 있습니다. 예제에서는 Bedrock의 `Anthropic Claude 3.7 Sonnet` 모델을 사용하겠습니다.

In [ ]:
model = BedrockModel(
    model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    # region_name="us-east-1",
    # boto_client_config=Config(
    #    read_timeout=900,
    #    connect_timeout=900,
    #    retries=dict(max_attempts=3, mode="adaptive"),
    # ),
    # temperature=0.9,
    # max_tokens=2048,
)

#### 내장 도구 가져오기

에이전트를 구축하는 다음 단계는 Strands Agents 내장 도구를 가져오는 것입니다. Strands Agents는 선택적 패키지 `strands-tools`에서 일반적으로 사용되는 내장 도구 세트를 제공합니다. 이 저장소에서 RAG, 메모리, 파일 작업, 코드 해석 등을 위한 도구를 사용할 수 있습니다. 예제에서는 `current_time` 도구를 사용하여 에이전트에 현재 시간에 대한 정보를 제공하고 `calculator` 도구를 사용하여 수학 계산을 수행합니다

In [ ]:
from strands_tools import calculator, current_time

#### 에이전트 정의

이제 필요한 모든 정보가 준비되었으므로 에이전트를 정의해보겠습니다

In [ ]:
agent = Agent(
    model=model,
    system_prompt=system_prompt,
    tools=[
        current_time,
        calculator,
        create_appointment,
        list_appointments,
        update_appointment,
    ],
)

## 에이전트 호출

이제 인사말로 레스토랑 에이전트를 호출하고 결과를 분석해보겠습니다

In [ ]:
results = agent("2+2는 얼마인가요?")

#### 에이전트 결과 분석

좋습니다! 처음으로 에이전트를 호출했습니다! 이제 결과 객체를 살펴보겠습니다. 먼저 에이전트 객체에서 에이전트가 교환하는 메시지를 볼 수 있습니다

In [ ]:
agent.messages

다음으로 결과 `metrics`를 분석하여 마지막 쿼리에 대한 에이전트 사용량을 살펴볼 수 있습니다

In [ ]:
results.metrics

#### 후속 질문으로 에이전트 호출
좋습니다. 이제 내일 약속을 잡아보겠습니다

In [ ]:
results = agent(
    "내일 오후 3시에 뉴욕에서 'Agent fun' 예약해주세요. 이 회의에서는 에이전트가 할 수 있는 모든 재미있는 일들에 대해 논의할 예정입니다"
)

#### 약속 업데이트

이제 이 약속을 업데이트해보겠습니다

In [ ]:
results = agent("아, 실수했네요! 'Agent fun'은 실제로 워싱턴 DC에서 열립니다")

#### 에이전트 결과 분석
에이전트 메시지와 결과 메트릭을 다시 살펴보겠습니다

In [ ]:
agent.messages

In [ ]:
results.metrics

#### 메시지에서 도구 사용량 확인

메시지 딕셔너리에서 도구 사용량을 자세히 살펴보겠습니다. 나중에 에이전트의 동작을 관찰하고 평가하는 방법을 보여드리겠지만, 이것이 그 방향의 첫 번째 단계입니다

In [ ]:
for m in agent.messages:
    for content in m["content"]:
        if "toolUse" in content:
            print("도구 사용:")
            tool_use = content["toolUse"]
            print("\tToolUseId: ", tool_use["toolUseId"])
            print("\t이름: ", tool_use["name"])
            print("\t입력: ", tool_use["input"])
        if "toolResult" in content:
            print("도구 결과:")
            tool_result = m["content"][0]["toolResult"]
            print("\tToolUseId: ", tool_result["toolUseId"])
            print("\t상태: ", tool_result["status"])
            print("\t내용: ", tool_result["content"])
            print("=======================")

### 작업이 올바르게 수행되었는지 검증
이제 데이터베이스를 확인하여 작업이 올바르게 수행되었는지 확인해보겠습니다. `Agent` 클래스는 `agent.tool.<tool_name>(<tool_params>)`를 호출하여 에이전트가 초기화된 도구를 직접 호출할 수 있는 기능을 제공합니다. 직접 도구 호출은 에이전트가 해당 도구를 직접 호출하지 않고도 도구에서 에이전트에게 정보를 제공하는 데 유용합니다. 이 직접 도구 호출을 사용하여 현재 약속을 나열할 수 있습니다:

In [ ]:
list_appointments_result = agent.tool.list_appointments()
print(json.dumps(list_appointments_result, indent=2))

도구 실행 결과가 `toolUseId`, 실행 `status`, 응답의 `content`를 포함하는 ToolResult 형식임을 알 수 있습니다. 다음과 같이 도구의 결과를 더 잘 시각화할 수 있습니다:

In [ ]:
list_appointments_result_text_content = list_appointments_result["content"][0]["text"]
print(json.dumps(json.loads(list_appointments_result_text_content), indent=2))

마지막으로, 직접 도구 호출을 사용하여 도구를 실행할 때 에이전트는 이러한 실행을 메시지 기록에 기록합니다. 기본적으로 이 기능이 활성화되어 있지만 `Agent` 클래스의 `record_direct_tool_call` 부울 플래그 속성으로 비활성화할 수 있습니다.

In [ ]:
current_time_result = agent.tool.current_time()
print("현재 시간 직접 도구 호출 결과:")
print(current_time_result)
current_time_direct_tool_messages = agent.messages[-4:]
print("현재 시간 직접 도구 호출 메시지:")
print(current_time_direct_tool_messages)

agent.record_direct_tool_call = False # record_direct_tool_call을 False로 설정
agent.tool.list_appointments()
after_disable_record_messages = agent.messages[-4:]
print("직접 도구 호출 기록을 비활성화한 후, 기록이 변경되지 않아야 합니다:")
print(current_time_direct_tool_messages == after_disable_record_messages)

## 확장: 확장된 사고
 
확장된 사고는 지원되는 Claude 계열 모델이 복잡한 작업에 대해 향상된 추론 기능을 활용할 수 있게 하여 최종 답변을 제공하기 전에 투명한 단계별 사고 과정을 제공합니다. 사고를 활성화하려면 Bedrock ModelProvider를 구성할 때 아래 구성을 포함할 수 있습니다. [확장된 사고에 대한 AWS 문서](https://docs.aws.amazon.com/bedrock/latest/userguide/claude-messages-extended-thinking.html)에서 자세히 알아볼 수 있습니다.

In [ ]:
thinking_model = BedrockModel(
    model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    additional_request_fields={
        "thinking": {
            "type": "enabled",
            "budget_tokens": 2048,
        }
    },
)

`thinking_model`을 정의한 후 새로운 `thinking_agent`를 생성하고 호출할 수 있습니다:

In [ ]:
thinking_system_prompt = """당신은 내 약속과 일정 관리를 전문으로 하는 도움이 되는 개인 비서입니다.
약속 관리 도구, 계산기에 액세스할 수 있으며 현재 시간을 확인하여 내 일정을 효과적으로 정리할 수 있습니다.
문제를 단계별로 생각하여 답을 도출합니다.
필요한 경우 업데이트할 수 있도록 항상 약속 ID를 제공해주세요"""

thinking_agent = Agent(
    model=thinking_model,
    system_prompt=thinking_system_prompt,
    tools=[
        current_time,
        calculator,
        create_appointment,
        list_appointments,
        update_appointment,
    ],
)

thinking_result = thinking_agent("내일 오후 2시에 새 약속을 추가하고 싶습니다")

에이전트의 메시지를 출력하여 확장된 사고 기능을 더 잘 분석할 수 있습니다. 확장된 사고는 에이전트의 응답에서 `reasoningContent` 블록으로 표현됩니다.

In [ ]:
thinking_agent.messages

## 훌륭한 작업입니다!
다음 모듈에서 만나요. :)